In [2]:
! pip install git+https://github.com/openai/whisper.git -q

In [3]:
import whisper

model = whisper.load_model("base")

In [9]:
! pip install gradio -q


In [10]:
import gradio as gr 
import time

In [11]:
def transcribe(audio):
    # Load the audio file
    audio = whisper.load_audio(audio)
    
    # Define chunk size (e.g., 30 seconds in samples, assuming 16000 Hz sample rate)
    chunk_size = 16000 * 30  # Whisper's sample rate is 16000 Hz
    
    # Split the audio into chunks
    chunks = [audio[i:i + chunk_size] for i in range(0, len(audio), chunk_size)]

    # Initialize the result text
    full_transcription = ""

    # Process each chunk
    for i, chunk in enumerate(chunks):
        # Pad or trim the chunk to the required length
        chunk = whisper.pad_or_trim(chunk)

        # Generate the log-Mel spectrogram
        mel = whisper.log_mel_spectrogram(chunk).to(model.device)

        # Detect the spoken language
        _, probs = model.detect_language(mel)
        print(f"Chunk {i + 1}: Detected language: {max(probs, key=probs.get)}")

        # Decode the chunk
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)

        # Append the transcription
        full_transcription += result.text + " "
        print(f"Chunk {i + 1} transcription: {result.text}")

    # Return the full transcription
    return full_transcription

In [ ]:
interface = gr.Interface(
    title="VOCAL-SENSE",
    # subtitle = "Decoding Speech, Summarizing Thoughts, and Detecting Emotions",
    fn=transcribe,
    inputs=[
        gr.Audio(sources=["microphone", "upload"], type="filepath", label="Record or Upload Audio")
    ],
    outputs=[
        gr.Textbox(label="Transcription")
    ],
    live=True
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


: 